<a href="https://colab.research.google.com/github/NARAEIM/tp2-da/blob/imnarae/high_low_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
click_df = pd.read_csv('C:/Users/lovel/Desktop/tp2/tp2-da/click_stream_final.csv')
#컬럼명 공백제거
click_df.columns = click_df.columns.str.strip()
click_df.head(1)

,session_id,event_name,event_time,traffic_source,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06 15:54:32.821085+00:00,MOBILE,NaN,0,0,NaN,NaN,0.00,0


In [ ]:
customer_df = pd.read_csv('C:/Users/lovel/Desktop/tp2/tp2-da/tp2_imnarae/customer_add_rfm.csv')
customer_df.head(1)

,customer_id,gender,birthdate,home_location,first_join_date,age_join,age_now,age_segment,RFM_Segment_x,join_year,join_month,RFM_Segment_y
0,2870,F,1996-06-14,Sumatera Barat,2019-07-21,23,26,Late 20s(26-30),NaN,2019,7,NaN


In [ ]:
transaction_df = pd.read_csv('C:/Users/lovel/Desktop/tp2/tp2-da/transaction_final.csv')
transaction_df.sort_values(by=['customer_id', 'created_at'], inplace=True)
transaction_df.head(1)

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,total_amount,product_id,quantity,item_price,with_promo
352123,2018-08-25 18:54:03.844494+00:00,3,856b0c3d-b076-406b-be11-3dfee762e9be,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,Credit Card,1,8073,6.00,0,826597,56019,5,166934,1


In [ ]:
# customer_id, booking_id 별 total_amount와 shipment_fee 차이
sales = pd.DataFrame(transaction_df.groupby(['customer_id', 'booking_id']).total_amount.unique() - transaction_df.groupby(['customer_id', 'booking_id']).shipment_fee.unique())
sales.head(1)

,,0
customer_id,booking_id,
3,0b3f17a4-f733-4eac-b9e9-37c039746e95,[174717]


In [ ]:
# 데이터프레임 재구조화
sales = sales.reset_index()

# 컬럼명 변경 (0->sales)
sales.rename(columns={0:'sales'}, inplace=True)

# sales 컬럼 형식 ([] -> int)
sales.sales = sales.sales.apply(lambda x:x[0])
sales.head()

,customer_id,booking_id,sales
0,3,0b3f17a4-f733-4eac-b9e9-37c039746e95,174717
1,3,0fb5a9e6-f306-4c37-b7f0-992bc78064a7,189939
2,3,1229bd35-7747-447a-a60b-02860bf7da7e,287523
3,3,163e447d-f2c2-4fac-b8a1-015eb01afd41,359032
4,3,182774df-3cde-46cf-b302-4d93e0d94288,200436


In [ ]:
#  'transaction_df','sales' 머지 (기준 : 'booking_id')
combined_df = pd.merge(sales, transaction_df[['booking_id', 'created_at']], on='booking_id')
combined_df.head(1)

,customer_id,booking_id,sales,created_at
0,3,0b3f17a4-f733-4eac-b9e9-37c039746e95,174717,2020-05-31 05:45:52.844494+00:00


In [ ]:
product_df = pd.read_csv('C:/Users/lovel/Desktop/tp2/tp2-da/product_final.csv')
product_df.head(1)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,usage,productDisplayName,brand
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,Casual,Turtle Check Men Navy Blue Shirt,turtle


In [ ]:
# 'product_id' 별 'item_price'의 고유값 개수를 계산
item_price_cnt = transaction_df.groupby('product_id')['item_price'].nunique()

# 'item_price'의 고유값 개수가 2개 이상인 'product_id'
multi_prices = item_price_cnt[item_price_cnt > 1]

# 'product_id'가 multiple_prices에 있는 모든 행을 추출
result_df = transaction_df[transaction_df['product_id'].isin(multi_prices.index)]

result_df.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,total_amount,product_id,quantity,item_price,with_promo
352123,2018-08-25 18:54:03.844494+00:00,3,856b0c3d-b076-406b-be11-3dfee762e9be,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,Credit Card,1,8073,6.00,0,826597,56019,5,166934,1
352124,2018-09-22 18:42:52.844494+00:00,3,f9b17de8-5db3-40dc-9227-664c6679543c,36673b2f-940a-42b5-994d-1cce4796b55f,Credit Card,1,0,0.00,10000,163784,30235,1,153784,0
352125,2018-10-20 19:07:54.844494+00:00,3,caa8be57-6239-4b5b-ae0b-e27887a13356,93e1d3c4-bed1-4ef4-b79f-e24a79e905b7,Credit Card,1,8828,3.00,10000,124932,7808,1,123760,1
352126,2018-11-17 19:09:49.844494+00:00,3,688cef52-8ee7-44ea-98af-20daf0c5609b,8356370c-267e-4530-a2d3-1830e4be3f2b,Credit Card,1,3415,3.00,0,247902,48968,1,251317,1
352127,2018-12-15 19:33:50.844494+00:00,3,a12fb778-d19a-4559-9808-8115f01c1595,ef7f4829-e7d1-4513-861b-1fbbc6e41bbf,Credit Card,1,2645,5.00,10000,181237,59272,1,173882,1


In [ ]:
print(transaction_df[['product_id', 'item_price']].isna().sum())
print(transaction_df[['product_id', 'item_price']].dtypes)

product_id    0
item_price    0
dtype: int64
product_id    int64
item_price    int64
dtype: object


In [ ]:
price_variation_df = transaction_df.groupby("product_id")["item_price"].nunique()
multiple_price_product_id = price_variation_df[price_variation_df > 1].index
multiple_price_products = transaction_df[transaction_df["product_id"].isin(multiple_price_product_id)]
multiple_price_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1254578 entries, 352123 to 1202276
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   created_at      1254578 non-null  object 
 1   customer_id     1254578 non-null  int64  
 2   booking_id      1254578 non-null  object 
 3   session_id      1254578 non-null  object 
 4   payment_method  1254578 non-null  object 
 5   payment_status  1254578 non-null  int64  
 6   promo_amount    1254578 non-null  int64  
 7   promo_code      1254578 non-null  float64
 8   shipment_fee    1254578 non-null  int64  
 9   total_amount    1254578 non-null  int64  
 10  product_id      1254578 non-null  int64  
 11  quantity        1254578 non-null  int64  
 12  item_price      1254578 non-null  int64  
 13  with_promo      1254578 non-null  int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 143.6+ MB


In [ ]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1254578 entries, 352123 to 1202276
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   created_at      1254578 non-null  object 
 1   customer_id     1254578 non-null  int64  
 2   booking_id      1254578 non-null  object 
 3   session_id      1254578 non-null  object 
 4   payment_method  1254578 non-null  object 
 5   payment_status  1254578 non-null  int64  
 6   promo_amount    1254578 non-null  int64  
 7   promo_code      1254578 non-null  float64
 8   shipment_fee    1254578 non-null  int64  
 9   total_amount    1254578 non-null  int64  
 10  product_id      1254578 non-null  int64  
 11  quantity        1254578 non-null  int64  
 12  item_price      1254578 non-null  int64  
 13  with_promo      1254578 non-null  int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 143.6+ MB


In [ ]:
transaction_df['product_id'].nunique()

44446

In [ ]:
product_df['id'].nunique()

44424

In [ ]:
#  'product_id'별 'item_price'상위 30%
top_30_prices = transaction_df.groupby('product_id')['item_price'].apply(lambda x: x.quantile(0.7)).reset_index(name='top_30_price')

# 'transaction_df', 'top_30_prices' 머지
high_merged = pd.merge(transaction_df, top_30_prices, on='product_id')

# 상위 30% 가격 이상으로 구매한 경우의 'customer_id'
high_price_customers = high_merged[high_merged['item_price'] >= high_merged['top_30_price']]['customer_id'].unique()



In [ ]:
len(high_price_customers)

40184

In [ ]:
# 'product_id'별 'item_price'하위 30%
low_30_prices = transaction_df.groupby('product_id')['item_price'].apply(lambda x: x.quantile(0.3)).reset_index(name='low_30_price')

# 'transaction_df', 'low_30_prices' 머지
low_merged = pd.merge(transaction_df, low_30_prices, on='product_id')

# 하위 30% 가격 이상으로 구매한 경우의 'customer_id'
low_price_customers = low_merged[low_merged['item_price'] >= low_merged['low_30_price']]['customer_id'].unique()

In [ ]:
len(low_price_customers)

47603

In [ ]:
# 상위 30% 가격에 해당하는 booking_id
high_price_booking = high_merged[high_merged['item_price'] >= high_merged['top_30_price']]['booking_id'].unique()



In [ ]:
# 하위 30% 가격에 해당하는 booking_id
low_price_booking = low_merged[low_merged['item_price'] <= low_merged['low_30_price']]['booking_id'].unique()



In [ ]:
# # sales_df에 새로운 컬럼 생성
# transaction_df['price_category'] = transaction_df['booking_id'].apply(lambda x: 'high' if x in high_price_booking else ('low' if x in low_price_booking else np.nan))

In [ ]:
high_price_df = pd.DataFrame(high_price_booking, columns=['booking_id'])
high_price_df['price_category'] = 'high'

low_price_df = pd.DataFrame(low_price_booking, columns=['booking_id'])
low_price_df['price_category'] = 'low'

price_category_df = pd.concat([high_price_df, low_price_df])


In [ ]:
#transaction_df2.info()

In [ ]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1254578 entries, 352123 to 1202276
Data columns (total 14 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   created_at      1254578 non-null  object 
 1   customer_id     1254578 non-null  int64  
 2   booking_id      1254578 non-null  object 
 3   session_id      1254578 non-null  object 
 4   payment_method  1254578 non-null  object 
 5   payment_status  1254578 non-null  int64  
 6   promo_amount    1254578 non-null  int64  
 7   promo_code      1254578 non-null  float64
 8   shipment_fee    1254578 non-null  int64  
 9   total_amount    1254578 non-null  int64  
 10  product_id      1254578 non-null  int64  
 11  quantity        1254578 non-null  int64  
 12  item_price      1254578 non-null  int64  
 13  with_promo      1254578 non-null  int64  
dtypes: float64(1), int64(9), object(4)
memory usage: 143.6+ MB


In [ ]:
high_price_df.head(1)

,booking_id,price_category
0,edaae189-4df1-4fd4-8e2f-e9de80803c62,high


In [ ]:
transaction_df.columns

Index(['created_at', 'customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'total_amount', 'product_id', 'quantity', 'item_price',
       'with_promo'],
      dtype='object')

In [ ]:
# 각 product_id별로 상위 30%, 하위 30%의 item_price를 계산
price_ranges = transaction_df.groupby('product_id')['item_price'].apply(lambda x: pd.Series({
    'low': x.quantile(0.3),
    'high': x.quantile(0.7)
})).reset_index()

In [ ]:
# transaction_df에 가격 범위를 병합
transaction_df = transaction_df.merge(price_ranges, on='product_id')

In [ ]:
transaction_df.columns

Index(['created_at', 'customer_id', 'booking_id', 'session_id',
       'payment_method', 'payment_status', 'promo_amount', 'promo_code',
       'shipment_fee', 'total_amount', 'product_id', 'quantity',
       'item_price_x', 'with_promo', 'level_1', 'item_price_y'],
      dtype='object')

In [ ]:
# # 각 행의 item_price가 해당 product_id의 상위 30% 또는 하위 30% 범위에 속하는지 확인
# transaction_df['price_category'] = np.where(
#     transaction_df['item_price_x'] >= transaction_df['leve'], 'high',
#     np.where(transaction_df['item_price_x'] <= transaction_df['low'], 'low', np.nan)
# )

In [ ]:
# price_ranges 데이터프레임 확인
print(price_ranges.head())

# 'transaction_df'와 'price_ranges' 병합 확인
transaction_df3 = transaction_df.merge(price_ranges, on='product_id', how='left')
print(transaction_df3.head())


   product_id level_1  item_price
0        1163     low   212885.40
1        1163    high   276118.10
2        1164     low   186561.80
3        1164    high   296005.40
4        1165     low   169517.90
                         created_at  customer_id  \
0  2018-08-25 18:54:03.844494+00:00            3   
1  2018-08-25 18:54:03.844494+00:00            3   
2  2018-08-25 18:54:03.844494+00:00            3   
3  2018-08-25 18:54:03.844494+00:00            3   
4  2022-02-21 17:31:24.928506+00:00         1234   

                             booking_id                            session_id  \
0  856b0c3d-b076-406b-be11-3dfee762e9be  f03b6825-1e6e-4916-bf71-f94ecbcbaee5   
1  856b0c3d-b076-406b-be11-3dfee762e9be  f03b6825-1e6e-4916-bf71-f94ecbcbaee5   
2  856b0c3d-b076-406b-be11-3dfee762e9be  f03b6825-1e6e-4916-bf71-f94ecbcbaee5   
3  856b0c3d-b076-406b-be11-3dfee762e9be  f03b6825-1e6e-4916-bf71-f94ecbcbaee5   
4  0ed0c3b1-c8a3-47c6-b612-65937e8e3d07  632940c9-3cc9-4743-b087-6213717518fc 

In [ ]:
# 'level_1' 컬럼을 pivot하여 'low'와 'high' 컬럼 생성
price_ranges = price_ranges.pivot(index='product_id', columns='level_1', values='item_price').reset_index()


In [ ]:
transaction_df3.head(30)

# transaction_df3['high'] : item_price기준 상위 30% 금액(해당 금액 이상이면 high)
# transaction_df3['low'] : item_price기준 하위 30% 금액(해당 금액 이하이면 low)
# transaction_df3['price_category'] : 해당 행의  item_price 가 high인지 low 인지

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,total_amount,product_id,quantity,item_price_x,with_promo,level_1,item_price_y,high,low
0,2018-08-25 18:54:03.844494+00:00,3,856b0c3d-b076-406b-be11-3dfee762e9be,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,Credit Card,1,8073,6.00,0,826597,56019,5,166934,1,low,171456.40,301043.60,171456.40
1,2018-08-25 18:54:03.844494+00:00,3,856b0c3d-b076-406b-be11-3dfee762e9be,f03b6825-1e6e-4916-bf71-f94ecbcbaee5,Credit Card,1,8073,6.00,0,826597,56019,5,166934,1,high,301043.60,301043.60,171456.40
2,2022-02-21 17:31:24.928506+00:00,1234,0ed0c3b1-c8a3-47c6-b612-65937e8e3d07,632940c9-3cc9-4743-b087-6213717518fc,OVO,1,0,0.00,10000,159050,56019,1,149050,0,low,171456.40,301043.60,171456.40
3,2022-02-21 17:31:24.928506+00:00,1234,0ed0c3b1-c8a3-47c6-b612-65937e8e3d07,632940c9-3cc9-4743-b087-6213717518fc,OVO,1,0,0.00,10000,159050,56019,1,149050,0,high,301043.60,301043.60,171456.40
4,2021-10-27 22:16:27.937313+00:00,2353,0ddfec6d-0277-40d2-b444-0aef8bdbec0b,4075da5d-9f93-4137-82cf-57a5022eadff,LinkAja,1,0,0.00,15000,194994,56019,1,179994,0,low,171456.40,301043.60,171456.40
5,2021-10-27 22:16:27.937313+00:00,2353,0ddfec6d-0277-40d2-b444-0aef8bdbec0b,4075da5d-9f93-4137-82cf-57a5022eadff,LinkAja,1,0,0.00,15000,194994,56019,1,179994,0,high,301043.60,301043.60,171456.40
6,2022-07-26 06:37:01.505360+00:00,3867,edaae189-4df1-4fd4-8e2f-e9de80803c62,daa49f51-7443-4520-b3d3-ed929cc04cf7,OVO,1,0,0.00,10000,311594,56019,1,301594,0,low,171456.40,301043.60,171456.40
7,2022-07-26 06:37:01.505360+00:00,3867,edaae189-4df1-4fd4-8e2f-e9de80803c62,daa49f51-7443-4520-b3d3-ed929cc04cf7,OVO,1,0,0.00,10000,311594,56019,1,301594,0,high,301043.60,301043.60,171456.40
8,2021-05-04 08:07:55.803619+00:00,5406,2256b519-5493-4da8-b9a2-14a536b1e66f,57468173-7105-4fa6-bacb-b7f875c8710d,OVO,1,0,0.00,10000,528073,56019,1,241854,0,low,171456.40,301043.60,171456.40
9,2021-05-04 08:07:55.803619+00:00,5406,2256b519-5493-4da8-b9a2-14a536b1e66f,57468173-7105-4fa6-bacb-b7f875c8710d,OVO,1,0,0.00,10000,528073,56019,1,241854,0,high,301043.60,301043.60,171456.40


In [ ]:
transaction_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2509156 entries, 0 to 2509155
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   created_at      object 
 1   customer_id     int64  
 2   booking_id      object 
 3   session_id      object 
 4   payment_method  object 
 5   payment_status  int64  
 6   promo_amount    int64  
 7   promo_code      float64
 8   shipment_fee    int64  
 9   total_amount    int64  
 10  product_id      int64  
 11  quantity        int64  
 12  item_price_x    int64  
 13  with_promo      int64  
 14  level_1         object 
 15  item_price_y    float64
 16  high            float64
 17  low             float64
dtypes: float64(4), int64(9), object(5)
memory usage: 344.6+ MB
